In [1]:
from data_extractor import DataExtractor

In [2]:
import h3

h3.__version__

'4.2.2'

In [3]:
extractor = DataExtractor()
# extractor.download_catalog(["precip"])

In [4]:
precip, tmin, tmax = extractor.process_catalog(["precip", "tmin", "tmax"])

In [5]:
_, state_polygons = extractor.aggregate_shape(precip)

c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [12]:
# _, state_polygons = extractor.aggregate_shape(precip)
precip_gdf_hex, hexagon_polygons = extractor.aggregate_h3_australia(precip, state_polygons, resolution=4)
# tmax_gdf_hex, _ = extractor.aggregate_h3_australia(tmax, state_polygons)
# tmin_gdf_hex, _ = extractor.aggregate_h3_australia(tmin, state_polygons)

c:\Monash\FIT3164\github_repo\2025_DataScienceProject2_DS07\data_gen\data_extractor.py:131: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:7844

  aggregated_gdf = gpd.sjoin(


In [20]:
precip_gdf, _ = extractor.aggregate_shape(precip)
tmax_gdf, _ = extractor.aggregate_shape(tmax)
tmin_gdf, _ = extractor.aggregate_shape(tmin)

c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [ ]:
state_polygons.to_csv("../../data/processed/processed_climate_data/suburb/suburb_geometries.csv")

In [ ]:
import pandas as pd
import calendar

def refactor_columns(df, metric):
    new_columns = {}
    for col in df.columns:
        if '_month_' in col:
            parts = col.split('_month_')
            year = parts[0]
            month_num = int(parts[1])
            month_name = calendar.month_name[month_num]
            new_col = f"{metric}_{year}_{month_name}"
            new_columns[col] = new_col
    
    df = df.rename(columns=new_columns)
    return df

precip_gdf = refactor_columns(precip_gdf, "precip")
tmin_gdf = refactor_columns(tmin_gdf, "tmin")
tmax_gdf = refactor_columns(tmax_gdf, "tmax")

In [ ]:
precip_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_precip_mm_suburb.csv")
tmin_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_tmin_suburb.csv")
tmax_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_tmax_suburb.csv")

In [ ]:
precip_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_precip_mm_suburb.csv")
tmin_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_tmin_suburb.csv")
tmax_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_tmax_suburb.csv")

In [ ]:
import geopandas as gpd

test = gpd.read_file("../../data/raw/SAL_2021_AUST_GDA2020.shp")

In [ ]:
test

In [ ]:
precip_gdf.columns

In [ ]:
test.shape

In [ ]:
precip_gdf.crs

In [ ]:
climate_gdf = gpd.GeoDataFrame(
            precip,
            geometry=gpd.points_from_xy(precip.lon, precip.lat),
            crs="EPSG:7844"
        )

In [ ]:
climate_gdf.crs

In [ ]:
def aggregate_shape(df):
        climate_gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df.lon, df.lat),
            crs="EPSG:7844"
        )

        suburbs_gdf = gpd.read_file("../../data/raw/SAL_2021_AUST_GDA2020.shp")

        climate_gdf = climate_gdf.to_crs(suburbs_gdf.crs)

        joined = gpd.sjoin(climate_gdf, suburbs_gdf, how="left", predicate="intersects")
        aggregated = joined.groupby("SAL_NAME21")[df.columns.to_list()[2:]].mean().reset_index()

        aggregated = aggregated.merge(
            suburbs_gdf[["SAL_NAME21", "geometry"]],
            on = "SAL_NAME21",
            how = "left"
        )

        return aggregated

In [ ]:
aggregate_shape(precip)

In [ ]:
unmatched_suburbs = test[test["SAL_NAME21"].isin(precip_gdf["SAL_NAME21"].unique())]

In [ ]:
unmatched_suburbs["SHAPE_Area"].describe()

In [ ]:
tmin_gdf[tmin_gdf["SAL_NAME21"].str.contains("Mount Buller")][[i for i in tmin_gdf.columns if "2020" in i]]

In [ ]:
state_polygons

In [ ]:
import folium
import geopandas
import geopandas as gpd

merged = state_polygons.merge(tmax_gdf, on="SAL_NAME21", how="inner")

merged = gpd.GeoDataFrame(merged, geometry="geometry", crs=state_polygons.crs)

m = merged[merged["STE_NAME21"] == "Victoria"].explore(
    column="tmax_2020_January",
    cmap="coolwarm",
    tooltip=["SAL_NAME21", "STE_NAME21", "tmax_2020_January"],
    popup=False,
    tiles="OpenStreetMap"
)

In [ ]:
m.save("../../data/processed/tmax_test_vis.html")

In [ ]:
precip_gdf["STE_NAME21"].unique()

In [22]:
import folium
import geopandas
import geopandas as gpd
merged = hexagon_polygons.merge(precip_gdf_hex, on="hex_id", how="inner")

merged = gpd.GeoDataFrame(merged, geometry="geometry", crs=state_polygons.crs)

m = merged.explore(
    column="2020_month_1",
    cmap="Blues",
    tooltip=["2020_month_1"],
    popup=False,
    tiles="OpenStreetMap"
)

In [14]:
merged

,hex_id,geometry,2000_month_1,2000_month_2,2000_month_3,2000_month_4,2000_month_5,2000_month_6,2000_month_7,2000_month_8,...,2019_month_8,2019_month_9,2019_month_10,2019_month_11,2019_month_12,2020_month_1,2020_month_2,2020_month_3,2020_month_4,2020_month_5
0,8494181ffffffff,"POLYGON ((125.36694 -14.00928, 125.33034 -14.2...",867.088013,454.851349,358.260590,189.604156,0.000000,0.000000,0.000000,0.000000,...,0.115461,0.180681,2.815063,3.247158,34.392666,390.576996,259.592987,185.280472,21.990774,24.313442
1,8494183ffffffff,"POLYGON ((125.77817 -14.09717, 125.74132 -14.3...",687.659058,432.509460,395.888855,160.125275,0.000000,0.000000,0.070158,0.000000,...,0.060838,0.078605,9.381511,2.869449,49.375256,393.659546,279.375519,167.103607,16.364960,15.356702
2,8494185ffffffff,"POLYGON ((125.10709 -14.29543, 125.07068 -14.5...",900.685913,454.094025,367.768097,199.135223,0.000000,0.000000,0.000000,0.000000,...,0.035670,0.442845,5.989515,7.980640,54.658459,417.313171,283.767883,177.775223,31.924309,41.946873
3,8494187ffffffff,"POLYGON ((125.51733 -14.38384, 125.48067 -14.6...",857.526733,440.750488,357.980133,175.603088,0.000000,0.000000,0.000000,0.000000,...,0.029434,0.327491,9.453673,5.350500,61.285755,396.500061,245.987686,161.828979,11.209978,25.460663
4,849418bffffffff,"POLYGON ((125.62755 -13.72129, 125.59077 -13.9...",700.191284,443.565033,392.694153,161.558121,0.000000,0.000000,0.098701,0.000000,...,0.173725,0.068377,0.947865,0.707699,16.445742,371.919952,222.186813,169.182602,20.113178,13.147272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,84d896dffffffff,"POLYGON ((145.31644 -41.27815, 145.37661 -41.0...",47.247417,70.529465,55.779736,77.514175,193.512619,135.456161,231.210037,97.447937,...,157.070938,130.342163,57.633720,142.944504,84.965988,71.859993,58.097557,104.607155,191.255798,134.711609
4916,84d8b25ffffffff,"POLYGON ((145.53013 -42.84398, 145.59091 -42.5...",84.777550,93.623489,100.546051,158.684265,301.344757,155.779312,252.165359,132.487671,...,266.119080,166.122345,110.052032,193.423126,148.741959,107.379471,115.408005,191.338882,294.619629,201.118149
4917,84d8b2dffffffff,"POLYGON ((145.73197 -43.25484, 145.79246 -43.0...",74.088623,91.496407,107.901398,192.292206,336.918518,182.439896,209.128098,96.102470,...,299.467346,173.875732,125.216148,235.062683,191.782166,104.983322,120.708939,170.415909,287.167450,225.111145
4918,84d8b65ffffffff,"POLYGON ((146.52955 -43.74646, 146.58788 -43.5...",74.464180,77.826233,87.090607,121.538948,249.426270,138.588882,181.993469,103.638374,...,200.949097,113.459969,84.964790,185.752960,122.815750,81.240334,90.481949,150.449753,223.118759,129.603256


In [23]:
m.save("../../data/processed/precip_hex_4_test.html")

In [21]:
precip_gdf

,SAL_NAME21,STE_NAME21,2000_month_1,2000_month_2,2000_month_3,2000_month_4,2000_month_5,2000_month_6,2000_month_7,2000_month_8,...,2019_month_8,2019_month_9,2019_month_10,2019_month_11,2019_month_12,2020_month_1,2020_month_2,2020_month_3,2020_month_4,2020_month_5
0,ACT Remainder - Belconnen,Australian Capital Territory,40.620117,14.391602,39.996582,70.638184,66.414062,32.481934,37.396973,89.483398,...,32.143066,33.141602,19.880859,18.190430,2.254883,30.913574,130.390137,102.692871,96.467773,37.420410
1,ACT Remainder - Booth,Australian Capital Territory,55.965233,22.888086,61.590038,44.683594,63.586720,29.917187,32.728516,79.452538,...,23.629297,34.893360,34.065624,22.712696,2.778711,30.914845,154.024811,97.015427,63.430470,29.823048
2,ACT Remainder - Canberra Central,Australian Capital Territory,36.138672,14.248047,42.500000,70.272461,65.291016,29.508789,34.451172,86.489258,...,23.493164,34.841797,20.686523,14.809570,1.534180,26.902344,107.571289,116.852539,90.837891,29.422852
3,ACT Remainder - Coree,Australian Capital Territory,55.027020,17.467447,47.145184,68.966469,93.311523,44.892254,51.988281,118.160805,...,51.960285,41.961262,27.174805,26.120443,3.972005,38.042316,147.418945,114.303383,126.132484,50.116863
4,ACT Remainder - Cotter River,Australian Capital Territory,71.344589,26.733189,56.476006,59.815708,118.447823,56.457310,74.344376,141.026718,...,62.146206,51.357422,38.066338,36.413643,7.432199,49.247070,175.923752,129.496719,139.166855,57.110352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15348,Zillmere,Queensland,54.918945,77.060547,60.086914,65.573242,65.015625,38.613281,26.109375,17.850586,...,6.782227,5.155273,78.956055,15.767578,94.325195,180.560547,427.239258,121.685547,16.081055,16.231445
15349,Zilzie,Queensland,59.039062,137.791992,24.620117,149.593750,72.275391,71.307617,18.400391,28.541992,...,6.473633,1.172852,56.633789,4.216797,17.896484,115.231445,279.637695,257.785156,44.611328,46.702148
15350,Zuccoli,Northern Territory,354.745117,600.490234,371.973633,281.534180,27.791992,0.450195,0.000000,0.285156,...,0.013672,0.063477,19.983398,96.129883,99.597656,489.993164,258.039062,234.922852,108.322266,2.404297
15351,Zumsteins,Victoria,7.642480,46.485645,8.877637,53.817284,63.739063,53.986134,82.406937,45.596092,...,69.726562,41.018456,32.284962,40.708496,6.897461,32.795509,59.877148,18.963673,83.226761,72.723732
